# Mission : Analyse des données & gestion des stocks

Notebook généré automatiquement — Phase 1 (agrégation & nettoyage) et Phase 2 (analyses pour le CODIR).

Structure :

1. Chargement des fichiers
2. Rapprochement via la table de liaison
3. Détection d'erreurs (≥8 contrôles) et propositions de correction
4. Nettoyage / suggestions de pipeline
5. Analyses synthétiques pour le CODIR : indicateurs clés, top produits, recommandations

_Toutes les étapes sont en français et prêtes à être adaptées selon retours._

In [2]:
# 1) Imports et chargement des fichiers
import pandas as pd
from pathlib import Path

erp = pd.read_csv('/mnt/data/erp.csv')
liaison = pd.read_csv('/mnt/data/liaison.csv')
web = pd.read_csv('/mnt/data/web.csv')

print('ERP shape:', erp.shape)
print('Liaison shape:', liaison.shape)
print('Web shape:', web.shape)

display(erp.head())
display(liaison.head())
display(web.head())


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/erp.csv'

## 2) Rapprochement des tables via la table de liaison

Approche :
- Normaliser les identifiants (strip, uppercase) avant jointure.
- Utiliser une jointure à gauche depuis la table web vers la table liaison puis vers ERP pour conserver ventes.

In [ ]:
# Normalisation et jointures
def norm(x):
    try:
        return str(x).strip().upper()
    except:
        return x

erp['ref_erp_norm'] = erp['reference'].apply(norm) if 'reference' in erp.columns else erp.iloc[:,0].astype(str).apply(norm)
liaison['ref_erp_norm'] = liaison.get('ref_erp', liaison.iloc[:,0]).astype(str).apply(norm)
liaison['ref_web_norm'] = liaison.get('ref_web', liaison.iloc[:,1] if liaison.shape[1]>1 else liaison.iloc[:,0]).astype(str).apply(norm)
web['ref_web_norm'] = web.get('sku', web.iloc[:,0]).astype(str).apply(norm)

# Jointure : web -> liaison -> erp
web_liaison = web.merge(liaison[['ref_web_norm','ref_erp_norm']], how='left', left_on='ref_web_norm', right_on='ref_web_norm')
full = web_liaison.merge(erp, how='left', left_on='ref_erp_norm', right_on='ref_erp_norm', suffixes=('_web','_erp'))

print('Jointure terminée. Dimensions resultantes:', full.shape)
display(full.head())


In [ ]:
# Normalisation et jointures
def norm(x):
    try:
        return str(x).strip().upper()
    except:
        return x

erp['ref_erp_norm'] = erp['reference'].apply(norm) if 'reference' in erp.columns else erp.iloc[:,0].astype(str).apply(norm)
liaison['ref_erp_norm'] = liaison.get('ref_erp', liaison.iloc[:,0]).astype(str).apply(norm)
liaison['ref_web_norm'] = liaison.get('ref_web', liaison.iloc[:,1] if liaison.shape[1]>1 else liaison.iloc[:,0]).astype(str).apply(norm)
web['ref_web_norm'] = web.get('sku', web.iloc[:,0]).astype(str).apply(norm)

# Jointure : web -> liaison -> erp
web_liaison = web.merge(liaison[['ref_web_norm','ref_erp_norm']], how='left', left_on='ref_web_norm', right_on='ref_web_norm')
full = web_liaison.merge(erp, how='left', left_on='ref_erp_norm', right_on='ref_erp_norm', suffixes=('_web','_erp'))

print('Jointure terminée. Dimensions resultantes:', full.shape)
display(full.head())


## 3) Détection d'erreurs (au moins 8 contrôles)

Voici une série de contrôles automatisés pour détecter problèmes de qualité de données :

1. Identifiants non appariés (web sans ref_erp)
2. Doublons dans ERP ou Web
3. Types incorrects (ex: quantité vendue non numérique)
4. Valeurs négatives (stock, quantités vendues, prix)
5. Prix manquants ou nuls
6. Totaux incohérents (ex: CA calculé vs CA déclaré si présent)
7. Champs texte anormaux (balises HTML, caractères non imprimables)
8. Date de vente/flux hors intervalle attendu (si colonne date présente)

Chaque contrôle ci-dessous donne un DataFrame d'exemples pour investigation.

In [ ]:
# 3) Contrôles automatisés
checks = {}

# 1. web sans liaison vers ERP
checks['web_sans_erp'] = full[full['ref_erp_norm'].isna()].copy()

# 2. doublons
checks['doublons_erp'] = erp[erp.duplicated(subset=['ref_erp_norm'], keep=False)] if 'ref_erp_norm' in erp.columns else pd.DataFrame()
checks['doublons_web'] = web[web.duplicated(subset=['ref_web_norm'], keep=False)] if 'ref_web_norm' in web.columns else pd.DataFrame()

# 3. types incorrects
def to_numeric_errors(s):
    try:
        conv = pd.to_numeric(s, errors='coerce')
        return s[conv.isna() & s.notna()]
    except:
        return s

candidate_cols = []
for df,name in [(web,'web'),(erp,'erp')]:
    for c in df.columns:
        if any(k in c.lower() for k in ['qty','quant','sold','vente','price','prix','stock']):
            candidate_cols.append((name,c))
for name,c in candidate_cols:
    df = web if name=='web' else erp
    bad = to_numeric_errors(df[c])
    checks[f'type_non_numerique__{name}__{c}'] = df.loc[df[c].isin(list(bad))] if not bad.empty else pd.DataFrame()

# 4. valeurs négatives pour stock, quantités, prix
for df,name in [(erp,'erp'),(web,'web')]:
    for c in df.columns:
        if any(k in c.lower() for k in ['qty','quant','sold','vente','stock','price','prix']):
            try:
                numeric = pd.to_numeric(df[c], errors='coerce')
                checks[f'valeurs_negatives__{name}__{c}'] = df.loc[numeric < 0]
            except:
                checks[f'valeurs_negatives__{name}__{c}'] = pd.DataFrame()

# 5. prix manquants ou nuls dans ERP
if 'price' in erp.columns or 'prix' in erp.columns:
    price_col = 'price' if 'price' in erp.columns else 'prix'
    checks['prix_missing_ou_zero'] = erp.loc[erp[price_col].isnull() | (pd.to_numeric(erp[price_col], errors='coerce')==0)]

# 6. totaux incohérents : si web contient 'total' ou 'revenue' on compare au calcul
if any(c.lower() in ['total','revenue','ca','montant'] for c in web.columns):
    for c in web.columns:
        if c.lower() in ['total','revenue','ca','montant']:
            if 'qty' in web.columns and 'price' in web.columns:
                calc = pd.to_numeric(web['qty'], errors='coerce') * pd.to_numeric(web['price'], errors='coerce')
                checks['totaux_incoherents'] = web.loc[(pd.to_numeric(web[c], errors='coerce').round(2) != calc.round(2)) & calc.notna()]
            break
else:
    checks['totaux_incoherents'] = pd.DataFrame()

# 7. texte anormal (balises html dans description)
for df,name in [(web,'web'),(erp,'erp')]:
    for c in df.columns:
        if any(k in c.lower() for k in ['desc','description','title','name']):
            checks[f'texte_html__{name}__{c}'] = df.loc[df[c].astype(str).str.contains('<[^>]+>', regex=True, na=False)]

# 8. dates hors intervalle (si colonne date présente)
import datetime
today = pd.Timestamp.today()
for df,name in [(web,'web'),(erp,'erp')]:
    for c in df.columns:
        if 'date' in c.lower():
            try:
                dates = pd.to_datetime(df[c], errors='coerce')
                checks[f'dates_future__{name}__{c}'] = df.loc[dates > (today + pd.Timedelta(days=1))]
            except:
                checks[f'dates_future__{name}__{c}'] = pd.DataFrame()

for k,v in checks.items():
    print(f"{k}: {len(v)} exemples")


### Exemples d'erreurs détectées — afficher quelques lignes pour investigation

In [ ]:
# Afficher jusqu'à 10 exemples pour chaque check (s'il y en a)
for k,v in checks.items():
    if len(v)>0:
        print('\n---', k, '---\n')
        display(v.head(10))


## 4) Propositions de correction / pipeline d'amélioration

Pour chaque type d'erreur ci-dessus, voici des suggestions concrètes à implémenter côté ERP / Wordpress / ETL :

- **Identifiants non appariés** : mettre en place un contrôle à l'import (validation de format), conserver la table de liaison à jour automatiquement via un script de rapprochement quotidien.
- **Doublons** : bloquer la création de références identiques, ajouter un UID interne immuable.
- **Types / formats** : valider types (prix numeric, qty integer) au niveau des formulaires et imports CSV, ajouter règles strictes dans l'ETL.
- **Valeurs négatives** : rejeter ou alerter les valeurs négatives pour stock/quantité/price à la source.
- **Prix manquants / nuls** : forcer champ obligatoire et workflow d'activation produit seulement si prix renseigné.
- **Texte/HTML** : nettoyer HTML sur description ou stocker contenu HTML dans un champ séparé (raw_html) et une version clean pour le reporting.
- **Dates incohérentes** : vérifier fuseau/format et rejeter dates dans le futur sauf si justifié.

En plus : créer un script de QA quotidien qui exécute les checks ci-dessus et crée une alerte Slack/email si des anomalies dépassent un seuil.


In [ ]:
# 4) Exemple de corrections appliquées automatiquement (non destructif: crée de nouvelles colonnes)
clean = full.copy()
# normaliser prix et quantités
for c in clean.columns:
    if any(k in c.lower() for k in ['price','prix']):
        clean[c+'_num'] = pd.to_numeric(clean[c], errors='coerce')
    if any(k in c.lower() for k in ['qty','quant','sold','vente','stock']):
        clean[c+'_num'] = pd.to_numeric(clean[c], errors='coerce').fillna(0).astype(int)

# marquer les enregistrements sans correspondance ERP pour revue
clean['a_valider_liaison'] = clean['ref_erp_norm'].isna()

display(clean.head())


## 5) Analyses synthétiques pour le CODIR (livrable concis)

Indicateurs proposés à présenter (slides 3-6 max) :

- **Vision agrégée** : nombre de produits, % produits sans liaison, % produits sans stock, CA total (si données ventes), marge moyenne (si coût dispo).
- **Alertes opérationnelles** : top 10 produits avec ventes élevées et stock faible (risque rupture), top 10 avec stock élevé et ventes faibles (sur-stock).
- **Qualité données** : nombre d'anomalies détectées par catégorie (8 contrôles ci-dessus).
- **Recommandations rapides** : actions court terme (corriger table de liaison, fixer règles d'imports), moyen terme (script QA, dashboard) et long terme (ERP/Shop intégrés).

Le code ci-dessous génère ces indicateurs automatiquement.

In [ ]:
# 5) Calculs d'indicateurs simples
df = clean.copy()

# nombre de produits uniques
n_products = df['ref_web_norm'].nunique()
pct_sans_liaison = df['a_valider_liaison'].mean() * 100

# CA si qty * price present (essayer de détecter colonnes)
qty_col = None
price_col = None
for c in df.columns:
    if 'qty' in c.lower() and c.endswith('_num'):
        qty_col = c
    if ('price' in c.lower() or 'prix' in c.lower()) and c.endswith('_num'):
        price_col = c

ca_total = None
if qty_col and price_col:
    df['ca_calc'] = df[qty_col] * df[price_col]
    ca_total = df['ca_calc'].sum()

print(f"Nombre de produits (web): {n_products}")
print(f"% produits sans liaison ERP: {pct_sans_liaison:.1f}%")
if ca_total is not None:
    print(f"CA calculé (approx): {ca_total:.2f}")
else:
    print('CA non calculable : colonnes qty/price manquantes')


In [ ]:
# Top produits à risque / sur-stock (si données qty & stock présentes)
# tenter de localiser colonnes de stock
stock_cols = [c for c in df.columns if 'stock' in c.lower() or 'qty' in c.lower() or 'quant' in c.lower()]
print('Colonnes candidates pour stock/qty:', stock_cols)

# Top 10 ventes (par ca_calc)
if 'ca_calc' in df.columns:
    top_ventes = df.groupby('ref_web_norm')['ca_calc'].sum().sort_values(ascending=False).head(10)
    print('\nTop 10 CA par produit (ref_web_norm):')
    display(top_ventes)

# Top risque rupture : ventes élevées / stock faible (si stock_col exists)
if stock_cols:
    stock_col = stock_cols[0]
    ventes_par_produit = df.groupby('ref_web_norm')[qty_col].sum() if qty_col else pd.Series(dtype=float)
    merged = pd.DataFrame({'ventes': ventes_par_produit, 'stock': df.groupby('ref_web_norm')[stock_col].first()})
    merged = merged.fillna(0)
    merged['ratio_ventes_stock'] = merged['ventes'] / (merged['stock'] + 1)
    top_risque = merged.sort_values('ratio_ventes_stock', ascending=False).head(10)
    print('\nTop 10 risque rupture (ventes/stock):')
    display(top_risque)


### Recommandations synthétiques (slide pour CODIR)

1. **Court terme (1-2 semaines)**
- Corriger les 20 premières références non appariées et anomalies critiques (prix nuls, valeurs négatives).
- Mettre en place contrôle à l'import CSV et script QA quotidien.

2. **Moyen terme (1-3 mois)**
- Automatiser la synchronisation table de liaison ou standardiser les identifiants.
- Dashboard KPI (stock, CA, rotations) pour le suivi hebdomadaire.

3. **Long terme**
- Intégration ERP-Shop (ou middleware) pour éviter double saisie.
- Refonte du process d'activation produit (workflow) pour garantir qualité des données.

Fin — notebook prêt à être adapté pour la présentation.